In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from prefect import Flow, Parameter, unmapped
import pandas as pd
from prefect.engine.executors import DaskExecutor
from crawto.ml_flow import data_cleaning_flow
from tinydb import TinyDB

In [5]:
input_df = pd.read_csv("data/house-prices-advanced-regression-techniques/train.csv")
test= pd.read_csv("data/house-prices-advanced-regression-techniques/test.csv")

In [6]:
executor = DaskExecutor()
data_cleaner = data_cleaning_flow.run(
    input_data= input_df, 
    problem="regression", 
    target = "SalePrice", 
    features = "infer",
    executor=executor
)

[2020-04-23 16:43:41,409] INFO - prefect.FlowRunner | Beginning Flow run for 'data_cleaning'
[2020-04-23 16:43:41,460] INFO - prefect.FlowRunner | Starting flow run.
[2020-04-23 16:43:43,626] INFO - prefect.TaskRunner | Task 'problem': Starting task run...
[2020-04-23 16:43:43,712] INFO - prefect.TaskRunner | Task 'problem': finished task run for task with final state: 'Success'
[2020-04-23 16:43:44,312] INFO - prefect.TaskRunner | Task 'features': Starting task run...
[2020-04-23 16:43:44,402] INFO - prefect.TaskRunner | Task 'features': finished task run for task with final state: 'Success'
[2020-04-23 16:43:44,483] INFO - prefect.TaskRunner | Task 'target': Starting task run...
[2020-04-23 16:43:44,523] INFO - prefect.TaskRunner | Task 'target': finished task run for task with final state: 'Success'
[2020-04-23 16:43:44,662] INFO - prefect.TaskRunner | Task 'input_data': Starting task run...
[2020-04-23 16:43:44,710] INFO - prefect.TaskRunner | Task 'input_data': finished task run f

c:\users\cwcol\projects\crawto\.venv\lib\site-packages\sklearn\preprocessing\_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[2020-04-23 16:43:47,477] INFO - prefect.TaskRunner | Task 'fit_yeo_johnson_transformer': finished task run for task with final state: 'Success'
[2020-04-23 16:43:47,566] INFO - prefect.TaskRunner | Task 'save_data': Starting task run...
[2020-04-23 16:43:47,566] INFO - prefect.TaskRunner | Task 'fit_target_encoder': Starting task run...
[2020-04-23 16:43:47,652] INFO - prefect.TaskRunner | Task 'save_data': finished task run for task with final state: 'Success'
[2020-04-23 16:43:47,813] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': Starting task run...
[2020-04-23 16:43:47,899] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': finished task run for task with final state: 'Success'
[2020-04-23 16:43:48,048] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': Starting task run...
[2020-04-23 16:43:48,150] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': finished task run for task with final state: 'Succes

In [7]:
data_cleaner.message

'All reference tasks succeeded.'

In [7]:
from prefect.engine.executors import DaskExecutor
from crawto.meta_model import MetaModel, meta_model_flow
meta = MetaModel("regression", TinyDB("db.json"), use_default_models=True)
models = meta.models
executor = DaskExecutor()
meta_model_run = meta_model_flow.run(
    train_data = "transformed_train.df",
    valid_data = "transformed_valid.df",
    train_target = "train_target.df",
    problem = "regression",
    models = models,
    tinydb = "db.json",
    executor = executor,
)

[2020-04-23 03:11:38,234] INFO - prefect.FlowRunner | Beginning Flow run for 'meta_model_flow'
[2020-04-23 03:11:38,252] INFO - prefect.FlowRunner | Starting flow run.
[2020-04-23 03:11:38,320] INFO - prefect.TaskRunner | Task 'tinydb': Starting task run...
[2020-04-23 03:11:38,348] INFO - prefect.TaskRunner | Task 'tinydb': finished task run for task with final state: 'Success'
[2020-04-23 03:11:38,355] INFO - prefect.TaskRunner | Task 'models': Starting task run...
[2020-04-23 03:11:38,361] INFO - prefect.TaskRunner | Task 'valid_data': Starting task run...
[2020-04-23 03:11:38,401] INFO - prefect.TaskRunner | Task 'train_data': Starting task run...
[2020-04-23 03:11:38,424] INFO - prefect.TaskRunner | Task 'problem': Starting task run...
[2020-04-23 03:11:38,428] INFO - prefect.TaskRunner | Task 'train_target': Starting task run...
[2020-04-23 03:11:38,450] INFO - prefect.TaskRunner | Task 'models': finished task run for task with final state: 'Success'
[2020-04-23 03:11:38,452] INF

e:\projects\crawto\.venv\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\projects\crawto\.venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:474: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent(
E:\projects\crawto\crawto\meta_model.py:45: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X=train_data, y=target)


[2020-04-23 03:11:39,215] INFO - prefect.TaskRunner | Task 'fit_model[6]': finished task run for task with final state: 'Success'
[2020-04-23 03:11:39,226] INFO - prefect.TaskRunner | Task 'fit_model': finished task run for task with final state: 'Mapped'
[2020-04-23 03:11:41,231] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


In [ ]:
with Flow("data_visualization") as flow:
    transformed_train_df = pd.read_feather("transformed_train.df")
    transformed_valid_df = pd.read_feather("transformed_valid.df")
    svd = fit_svd(transformed_train_df)
    svd_train = svd_transform(svd, transformed_train_df, "transformed_train_df",tinydb)
    svd_valid = svd_transform(svd, transformed_valid_df, "transformed_valid_df",tinydb)
    